In [4]:
!pip install opencv-python

In [8]:
import cv2
print(cv2.__version__)

4.11.0


In [1]:
import cv2
import os
import pandas as pd
from shutil import copyfile

# Define input and output paths
input_dirs = [
    '/kaggle/input/qrcodes/QR_Dataset/test_images',
    '/kaggle/input/qrcodes/QR_Dataset/train_images'
]
output_dir = '/kaggle/working/combined_images/'  # Combined images directory
output_drawn_dir = '/kaggle/working/output_drawn_images/'  # Directory for images with bounding boxes
csv_path = '/kaggle/working/detected_qrcodes.csv'  # Path for CSV output

# Create output directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_drawn_dir, exist_ok=True)

print("Directories set up successfully.")

Directories set up successfully.


In [2]:
# Combine images from test and train folders into a single directory
for input_dir in input_dirs:
    for image_name in os.listdir(input_dir):
        if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter image files
            src_path = os.path.join(input_dir, image_name)
            dst_path = os.path.join(output_dir, image_name)
            copyfile(src_path, dst_path)
            print(f"Copied: {image_name} to {output_dir}")

print(f"All images combined into: {output_dir}")

Copied: img224.jpg to /kaggle/working/combined_images/
Copied: img212.jpg to /kaggle/working/combined_images/
Copied: img243.jpg to /kaggle/working/combined_images/
Copied: img210.jpg to /kaggle/working/combined_images/
Copied: img234.jpg to /kaggle/working/combined_images/
Copied: img204.jpg to /kaggle/working/combined_images/
Copied: img217.jpg to /kaggle/working/combined_images/
Copied: img206.jpg to /kaggle/working/combined_images/
Copied: img244.jpg to /kaggle/working/combined_images/
Copied: img247.jpg to /kaggle/working/combined_images/
Copied: img242.jpg to /kaggle/working/combined_images/
Copied: img229.jpg to /kaggle/working/combined_images/
Copied: img205.jpg to /kaggle/working/combined_images/
Copied: img220.jpg to /kaggle/working/combined_images/
Copied: img235.jpg to /kaggle/working/combined_images/
Copied: img246.jpg to /kaggle/working/combined_images/
Copied: img237.jpg to /kaggle/working/combined_images/
Copied: img209.jpg to /kaggle/working/combined_images/
Copied: im

In [3]:
# Initialize QR code detector
detector = cv2.QRCodeDetector()

# List to collect detected data
detected_data = []

# Loop through all images in the combined directory
for image_name in os.listdir(output_dir):
    if image_name.lower().endswith('.jpg'):  # Filter image files
        image_path = os.path.join(output_dir, image_name)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to load image: {image_name}")
            continue
        
        # Detect and decode multiple QR codes
        try:
            retval, decoded_info, points, _ = detector.detectAndDecodeMulti(img)
        except Exception as e:
            print(f"Error detecting QR codes in {image_name}: {e}")
            continue
        
        # Copy the image for drawing
        drawn_img = img.copy()
        
        if retval and points is not None:
            for i in range(len(points)):
                pts = points[i].astype(int)  # Points are the four corners of the QR code
                
                # Approximate bounding box (axis-aligned)
                x_min = min(pts[:, 0])
                x_max = max(pts[:, 0])
                y_min = min(pts[:, 1])
                y_max = max(pts[:, 1])
                
                # Draw green rectangle around the QR code
                cv2.rectangle(drawn_img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                
                # Collect data
                value = decoded_info[i] if decoded_info[i] else "Undecoded"
                detected_data.append({
                    'image_id': image_name,
                    'x_min': x_min,
                    'y_min': y_min,
                    'x_max': x_max,
                    'y_max': y_max,
                    'value': value
                })
        else:
            print(f"No QR codes detected in: {image_name}")
        
        # Save the drawn image (even if no QR detected, for manual review)
        output_image_path = os.path.join(output_drawn_dir, image_name)
        cv2.imwrite(output_image_path, drawn_img)
        print(f"Processed and saved: {image_name}")

# Save detected data to CSV
if detected_data:
    df = pd.DataFrame(detected_data)
    df.to_csv(csv_path, index=False)
    print(f"CSV saved to: {csv_path}")
else:
    print("No QR codes detected in any images.")

print("QR code detection and processing complete.")

Processed and saved: img173.jpg
Processed and saved: img241.jpg
Processed and saved: img049.jpg
Processed and saved: img004.jpg
Processed and saved: img148.jpg
No QR codes detected in: img112.jpg
Processed and saved: img112.jpg
Processed and saved: img070.jpg
Processed and saved: img043.jpg
Processed and saved: img051.jpg
Processed and saved: img166.jpg
Processed and saved: img087.jpg
Processed and saved: img124.jpg
Processed and saved: img121.jpg
Processed and saved: img170.jpg
No QR codes detected in: img010.jpg
Processed and saved: img010.jpg
Processed and saved: img177.jpg
Processed and saved: img174.jpg
Processed and saved: img132.jpg
Processed and saved: img152.jpg
Processed and saved: img008.jpg
No QR codes detected in: img131.jpg
Processed and saved: img131.jpg
No QR codes detected in: img130.jpg
Processed and saved: img130.jpg
No QR codes detected in: img129.jpg
Processed and saved: img129.jpg
Processed and saved: img029.jpg
Processed and saved: img199.jpg
Processed and saved:

In [5]:
import shutil
import os

# Define the folder or file to zip (adjust paths if needed)
folder_to_zip = '/kaggle/working/output_drawn_images'  # For the images folder
# Or for the CSV: file_to_zip = '/kaggle/working/detected_qrcodes.csv'

# Create a zip file of the folder
zip_filename = 'drawn_images.zip'  # Name your zip file
shutil.make_archive(zip_filename, 'zip', folder_to_zip)

# For a single file, zip it like this:
# shutil.make_archive('detected_qrcodes', 'zip', '/kaggle/working', 'detected_qrcodes.csv')

print(f"Zip file created: /kaggle/working/{zip_filename}")
print("Now go to the Output tab in the right sidebar, find the zip file, and download it using the three dots menu.")

Zip file created: /kaggle/working/drawn_images.zip
Now go to the Output tab in the right sidebar, find the zip file, and download it using the three dots menu.
